# Tutorial 5: Diagnostics and Visualization

This tutorial covers **diagnostic tools** for GMM/SMM estimation - how to visualize results, check model fit, and diagnose potential issues.

## What You'll Learn

1. Visualizing the objective function landscape
2. Checking moment fit
3. Diagnosing identification issues
4. Analyzing optimization convergence
5. Using the J-test for model specification

## Prerequisites

- Completed Tutorials 1-4

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from momentest import (
    smm_estimate,
    gmm_estimate,
    SMMEngine,
    GMMEngine,
    EstimationSetup,
    estimate,
    j_test,
    linear_iv,
    load_econ381,
    # Visualization functions
    plot_objective_landscape,
    plot_moment_contributions,
    plot_identification,
    plot_marginal_objective,
    plot_moment_comparison,
    plot_convergence,
    summary,
)

np.random.seed(42)

## 1. Setup: Truncated Normal Example

We'll use the truncated normal example for most visualizations:

In [ ]:
import scipy.stats as sts

# Load data
dataset = load_econ381()
data = dataset['data']
cut_lb, cut_ub = dataset['bounds']
N = len(data)

# Target moments
data_mean = data.mean()
data_var = data.var()
data_moments = np.array([data_mean, data_var])

# Simulation functions
def trunc_norm_draws(unif_vals, mu, sigma, cut_lb, cut_ub):
    sigma = max(sigma, 1e-6)
    cut_lb_cdf = sts.norm.cdf(cut_lb, loc=mu, scale=sigma)
    cut_ub_cdf = sts.norm.cdf(cut_ub, loc=mu, scale=sigma)
    cdf_range = cut_ub_cdf - cut_lb_cdf
    if cdf_range < 1e-10:
        return np.full_like(unif_vals, (cut_lb + cut_ub) / 2)
    unif_scaled = unif_vals * cdf_range + cut_lb_cdf
    unif_scaled = np.clip(unif_scaled, cut_lb_cdf + 1e-10, cut_ub_cdf - 1e-10)
    return np.clip(sts.norm.ppf(unif_scaled, loc=mu, scale=sigma), cut_lb, cut_ub)

def sim_func(theta, shocks):
    mu, sigma = theta
    sigma = max(sigma, 1.0)
    return trunc_norm_draws(shocks, mu, sigma, cut_lb, cut_ub)

def moment_func(sim_data):
    return np.column_stack([np.mean(sim_data, axis=1), np.var(sim_data, axis=1)])

# Create engine
engine = SMMEngine(
    k=2, p=2, n_sim=300, shock_dim=N,
    sim_func=sim_func, moment_func=moment_func, seed=42
)

# Estimate
result = smm_estimate(
    sim_func=sim_func, moment_func=moment_func,
    data_moments=data_moments.tolist(),
    bounds=[(0, 1000), (1, 500)],
    n_sim=300, shock_dim=N, seed=42, weighting="optimal",
)

print(f"Estimates: μ={result.theta[0]:.2f}, σ={result.theta[1]:.2f}")

## 2. Objective Function Landscape

Visualizing the objective function helps understand:
- Is there a unique minimum?
- Are there local minima?
- How "flat" is the objective near the minimum?

In [ ]:
# Get weighting matrix
_, S = engine.moments(result.theta)
W = np.linalg.inv(S)

# Plot objective landscape
fig = plot_objective_landscape(
    engine=engine,
    theta_hat=result.theta,
    data_moments=data_moments,
    W=W,
    param_indices=(0, 1),
    param_names=["μ", "σ"],
    n_points=40,
    scale=0.3,
    plot_type="both",
)
plt.show()

### Interpreting the Landscape

- **Well-identified**: Clear, unique minimum (bowl shape)
- **Poorly identified**: Flat regions, multiple minima, or ridges
- **Red star**: The estimated parameters

## 3. Marginal Objective Functions

Look at how the objective changes when varying one parameter at a time:

In [ ]:
# Plot marginal objectives
fig = plot_marginal_objective(
    engine=engine,
    theta_hat=result.theta,
    data_moments=data_moments,
    W=W,
    param_names=["μ", "σ"],
    n_points=50,
    scale=0.3,
)
plt.show()

## 4. Moment Contributions

See how each moment contributes to the objective:

In [ ]:
# Plot moment contributions
fig = plot_moment_contributions(
    engine=engine,
    theta_hat=result.theta,
    data_moments=data_moments,
    W=W,
    param_index=0,  # Vary μ
    param_names=["μ", "σ"],
    moment_names=["Mean", "Variance"],
    n_points=50,
    scale=0.3,
)
plt.show()

### Interpreting Moment Contributions

- **Informative moments**: Large contribution, steep slope
- **Uninformative moments**: Small contribution, flat
- **At estimate**: All contributions should be near zero

## 5. Identification Analysis

The Jacobian matrix $D = \partial m / \partial \theta$ shows which moments identify which parameters:

In [ ]:
# Plot identification (Jacobian heatmap)
fig = plot_identification(
    engine=engine,
    theta_hat=result.theta,
    param_names=["μ", "σ"],
    moment_names=["Mean", "Variance"],
)
plt.show()

### Interpreting the Jacobian

- **Left panel**: Raw Jacobian values
- **Right panel**: Normalized sensitivity (which moments respond most to each parameter)
- **Good identification**: Each parameter affects at least one moment strongly
- **Poor identification**: A column is all zeros (parameter doesn't affect any moment)

## 6. Moment Fit Comparison

In [ ]:
# Plot moment comparison
fig = plot_moment_comparison(
    data_moments=data_moments,
    model_moments=result.sim_moments,
    moment_names=["Mean", "Variance"],
)
plt.suptitle("Data vs Model Moments", y=1.02)
plt.show()

## 7. Full Summary Output

In [ ]:
# Print comprehensive summary
print(summary(
    theta=result.theta,
    se=result.se,
    objective=result.objective,
    data_moments=data_moments,
    model_moments=result.sim_moments,
    k=2,
    p=2,
    n=300,
    converged=result.converged,
    param_names=["μ", "σ"],
    moment_names=["Mean", "Variance"],
    method="SMM",
))

## 8. J-Test for Overidentification

Let's use a GMM example with overidentification to demonstrate the J-test:

In [ ]:
# Generate IV data
dgp = linear_iv(n=500, seed=42, beta0=1.0, beta1=2.0, rho=0.5)
dgp.data['Z2'] = dgp.data['Z']**2
dgp.data['Z3'] = dgp.data['Z']**3

def moment_func_gmm(data, theta):
    beta0, beta1 = theta
    residual = data['Y'] - beta0 - beta1 * data['X']
    return np.column_stack([
        residual,
        residual * data['Z'],
        residual * data['Z2'],
        residual * data['Z3'],
    ])

# Estimate
result_gmm = gmm_estimate(
    data=dgp.data,
    moment_func=moment_func_gmm,
    bounds=[(-10, 10), (-10, 10)],
    k=4,
    weighting="optimal",
)

print(f"GMM estimates: β₀={result_gmm.theta[0]:.4f}, β₁={result_gmm.theta[1]:.4f}")
print(f"True values:   β₀={dgp.true_theta[0]:.4f}, β₁={dgp.true_theta[1]:.4f}")

In [ ]:
# J-test
j_result = j_test(
    objective=result_gmm.objective,
    n=dgp.n,
    k=4,
    p=2,
)

print(j_result)

### Interpreting the J-Test

- **H₀**: All moment conditions are valid
- **Fail to reject** (high p-value): Model is correctly specified
- **Reject** (low p-value): At least one moment condition is invalid

Common reasons for rejection:
1. Invalid instruments (exclusion restriction violated)
2. Model misspecification
3. Heteroskedasticity not accounted for

## 9. Convergence Diagnostics

Let's look at how the optimization progressed:

In [ ]:
# Run estimation with history tracking
setup = EstimationSetup(
    mode="SMM", model_name="truncated_normal", moment_type="mean_var",
    k=2, p=2, n_sim=300, shock_dim=N, seed=42, weighting="optimal"
)

result_with_history = estimate(
    setup=setup,
    data_moments=data_moments,
    bounds=[(0, 1000), (1, 500)],
    n_global=50,
    engine=engine,
)

print(f"Total evaluations: {result_with_history.n_evals}")
print(f"History length: {len(result_with_history.history)}")

In [ ]:
# Plot convergence
fig = plot_convergence(
    engine=engine,
    history=result_with_history.history,
    data_moments=data_moments,
    W=result_with_history.W,
    param_indices=(0, 1),
    param_names=["μ", "σ"],
    n_points=40,
    scale=0.3,
)
plt.show()

## 10. Diagnostic Checklist

When estimating a model, check:

### Before Estimation
- [ ] Moments are well-defined and finite
- [ ] Parameter bounds are reasonable
- [ ] Enough simulations (n_sim ≥ 100)

### After Estimation
- [ ] Optimization converged (`result.converged == True`)
- [ ] Objective is small (moments are matched)
- [ ] Standard errors are finite and reasonable
- [ ] J-test doesn't reject (if overidentified)

### Visualization Checks
- [ ] Objective landscape has clear minimum
- [ ] No flat regions (identification)
- [ ] Moments are well-matched
- [ ] Convergence path is smooth

## 11. Exercises

### Exercise 1: Weak Identification
Create a model where one parameter is weakly identified. What do the diagnostics show?

### Exercise 2: Model Misspecification
Fit a truncated normal to data from a different distribution. Does the J-test detect it?

### Exercise 3: Multiple Local Minima
Create a model with multiple local minima. How does the objective landscape look?

In [ ]:
# Exercise 2 starter: Generate data from a different distribution
# and try to fit truncated normal

# Generate from beta distribution (not normal!)
np.random.seed(42)
data_beta = 450 * np.random.beta(2, 5, size=161)  # Skewed!

# Compute moments
data_moments_beta = np.array([data_beta.mean(), data_beta.var()])

# Fit truncated normal
result_misspec = smm_estimate(
    sim_func=sim_func, moment_func=moment_func,
    data_moments=data_moments_beta.tolist(),
    bounds=[(0, 1000), (1, 500)],
    n_sim=300, shock_dim=161, seed=42, weighting="optimal",
)

print(f"Fitted to beta data: μ={result_misspec.theta[0]:.2f}, σ={result_misspec.theta[1]:.2f}")
print(f"Objective: {result_misspec.objective:.4f}")
print("\nThe model fits the mean and variance, but the shape is wrong!")

## Summary

In this tutorial, you learned:

1. **Objective landscape**: Visualize the optimization surface
2. **Moment contributions**: See which moments drive the fit
3. **Identification**: Check the Jacobian for weak identification
4. **Moment fit**: Compare data vs model moments
5. **J-test**: Test overidentifying restrictions
6. **Convergence**: Track optimization progress

### Key Takeaways

- Always visualize the objective landscape
- Check identification before trusting estimates
- Use J-test for overidentified models
- Diagnostics help catch problems early

### Next Steps

- **Tutorial 6**: Advanced structural models (DDC, dynamic models)